# Grafi

Graf $G$ je abstraktni podatkovni tip, ki ga sestavljata množica **vozlišč** (*nodes, vertices, points*) $V$ in množica **povezav** (*edges, links*) $E$, ki predstavljajo relacije med pari vozlišč. Vozliščema, ki sestavljata povezavo, rečemo **krajišči** (*endpoints*). Vozlišča in povezave lahko hranijo tudi kakšne dodatne lastnosti.

<img alt="Graf" src="graph.svg" width="300px"/>

Običajne operacije, ki jih želimo izvajati na grafu so:
- dodajanje/odstranjevanje vozlišča/povezave
- nastavljanje/ugotavljanje lastnosti vozlišča/povezave
- ugotavljanje sosednosti dveh vozlišč
- iskanje vseh sosednjih vozlišč
- ...

Kadar z grafom modeliramo nek resničen pojav ali proces, namesto grafa pogosto uporabimo izraz *omrežje (network)*. Grafe lahko uporabimo za modeliranje številnih procesov, kot so razna družbena ali komunikacijska omrežja, omrežja soavtorstev ali celo biološka omrežja, ki modelirajo razne kemijske procese. Mi pa se bomo ukvarjali samo s strukturami brez njihovega ozadja, torej z grafi.

## Terminologija

Glavni lastnosti grafa sta število vozlišč $n = |V|$ in število povezav $e = |E|$ (za število povezav bomo včasih uporabljali tudi $m$).

Poznamo več vrst grafov glede na njihove lastnosti:

- **Neusmerjeni** (*undirected*) grafi vsebujejo same neusmerjene povezave, ki predstavljajo simetrične relacije, kjer vrstni red krajišč ni pomemben, npr. med dvema bratoma. **Usmerjeni** (*directed*) grafi (*digraphs*) pa so sestavljeni iz usmerjenih povezav, ki predstavljajo asimetrično relacijo, npr. od otroka k staršu. Te običajno ponazorimo z puščicami.
- Glede na lastnost povezav ločimo med **neuteženimi** (*unweighted*) in **uteženimi** (*weighted*) grafi. V neuteženih grafih so vse povezave enakovredne, v uteženih pa vsaki povezavi priredimo neko numerično vrednost, ki ji rečemo utež, in lahko predstavlja npr. dolžino, ceno, ...
- **Enostavni** (*simple*) grafi ne vsebujejo **zank** (*loop*), ki povezujejo vozlišče s samim seboj, in **vzporednih povezav** (*multiple/parallel edges*) med istimi pari vozlišč.
- Glede na prisotnost ciklov v grafih poznamo **aciklične** (*acyclic*) in **ciklične** (*cyclic*) grafe.
- Grafe precej grobo ločujemo tudi po razmerju med številom povezav in številom vozlišč. V **gostih** (*dense*) grafih je število vozlišč velikostnega reda, ki je blizu maksimalnemu številu možnih povezav, $e = O(n^2)$. V **redkih** (*sparse*) grafih pa je število povezav linearno odvisno od števila vozlišč $e = O(n)$.

Oglejmo si še nekaj drugih terminov povezanih z grafi:

- Tako kot pri drevesih, tudi v grafih poznamo **stopnjo** (*degree*) vozlišča, ki je enaka številu povezav, ki vključujejo to vozlišče. Če govorimo o stopnji grafa (kar bomo označevali z $d$), pa mislimo največjo stopnjo njegovega vozlišča. V usmerjenih grafih ločujemo **vhodno** in **izhodno** stopnjo (*indegree/outdegree*), ki sta število povezav, ki kažejo v vozlišče oz. izven njega.
- Dve vozlišči sta **sosednji** (*adjacent*) oz. **soseda**, če ju povezuje katera izmed povezav v grafu. Množici sosednjih vozlišč izbranega vozlišča rečemo tudi soseščina (*neighbourhood*).

Poleg že omenjenih splošnih vrst grafov, poznamo tudi več razredov grafov, ki imajo podobne strukturne lastnosti. Poznamo:

- **drevesa** (*trees*), ki so v kontekstu novih terminov pravzaprav aciklični povezani neusmerjeni graf
- **polne grafe** (*complete graph*), ki vsebujejo vse možne povezave
- **regularne grafe** (*regular graph*), v katerih imajo vsa vozlišča enako stopnjo
- **dvodelne grafe** (*bipartite graph*), ki so sestavljeni iz dveh skupin vozlišč, povezave pa potekajo samo med obema skupinama
- ...

Na grafih nas pogosto zanimajo premiki med sosednjimi vozlišči:

- **Sprehod** (*walk*) je poljubno zaporedje vozlišč, med katerimi se premikamo po povezavah v grafu. Če obstaja sprehod med dvema vozliščema, bomo rekli, da sta **povezani**. Spomnimo se, da če sta povezani neposredno z eno samo povezavo, jima rečemo tudi sosednji.
- **Obhod** (*closed walk*) je sprehod, ki se začne in konča v istem vozlišču.
- **Steza** (*trail*) je sprehod brez ponovljenih povezav.
- **Pot** (*path*) je sprehod brez ponovljenih vozlišč.
- **Cikel** (*cycle*) je obhod brez ponovljenih vmesnih vozlišč (z izjemo začetnega in končnega, ki sta enaka).

## Predstavitve

Strukturo grafa, ki jo definirajo vozlišča in povezave, moramo nekako predstaviti oz. shraniti, da bomo lahko na njej izvajali kakšen izračune. Glede na funkcionalnost, ki jo potrebujemo, poznamo tri pogoste načine predstavitve grafov. Če je treba, pa si lahko pomagamo kar z več različnimi predstavitvami sočasno.

In [1]:
#include <iostream>
#include <fstream>
#include <vector>
#include <queue>
#include <algorithm>
using namespace std;

typedef pair<int,int> PII;
typedef vector<int> VI;
typedef vector<pair<int,int>> VII;
typedef vector<vector<int>> VVI;

In [2]:
template<typename T>
void print(const vector<T> &sez) {
    for (T x : sez) cout << x << " ";
    cout << endl;
}

- **Seznam povezav** (*edge list*) je najbolj enostavna predstavitev. Vse povezave v grafu preprosto shranimo v seznam. Ta predstavitev bo primerna, če želimo obravnavati vse povezave ne glede na vrstni red.

In [3]:
VII read_graph(string fname, int &n, int &m) {
    ifstream fin(fname);
    fin >> n >> m;
    vector<PII> povezave;
    for (int i=0;i<m;i++) {
        int a,b;
        fin >> a >> b;
        povezave.push_back({a,b});
    }
    fin.close();
    return povezave;
}

In [4]:
int n,m;
vector<PII> povezave = read_graph("graph.txt",n,m);
for (auto [a,b] : povezave) cout << '(' << a << ',' << b << ')' << ' ';
cout << endl;

(0,1) (0,4) (1,3) (1,4) (1,5) (1,7) (2,3) (2,5) (4,5) (6,7) 


- **Seznam sosedov** (*adjacency list*) hrani za vsako vozlišče seznam njegovih sosedov. Kadar se premikamo po grafih od enega vozlišča k drugemu, nam to pride zelo prav.

In [5]:
VVI adjacency_list(VII &edge_list, int n, bool dir=false) {
    vector<VI> adj(n);
    for (auto [a,b] : edge_list) {
        adj[a].push_back(b);
        if (!dir) adj[b].push_back(a);
    }
    return adj;
}

In [6]:
vector<VI> sosedi = adjacency_list(povezave, n);
for (int i=0;i<n;i++) {
    cout << i << ": ";
    print(sosedi[i]);
}

0: 1 4 
1: 0 3 4 5 7 
2: 3 5 
3: 1 2 
4: 0 1 5 
5: 1 2 4 
6: 7 
7: 1 6 


- **Matrika sosednosti** (*adjacency matrix*) je namenjena učinkovitemu preverjanju sosednosti dveh vozlišč. Sestavimo namreč matriko $M$, kjer na mestu $M_{x,y}$ hranimo informacijo o prisotnosti ali teži povezave med vozliščema $x$ in $y$.

In [7]:
VVI adjacency_matrix(VII &edge_list, int n) {
    vector<VI> mat(n, vector<int>(n));
    for (auto [a,b] : edge_list) {
        mat[a][b] = 1;
        mat[b][a] = 1;
    }
    return mat;
}

In [8]:
vector<VI> sosednost = adjacency_matrix(povezave, n);
for (int i=0;i<n;i++) {
    print(sosednost[i]);
}

0 1 0 0 1 0 0 0 
1 0 0 1 1 1 0 1 
0 0 0 1 0 1 0 0 
0 1 1 0 0 0 0 0 
1 1 0 0 0 1 0 0 
0 1 1 0 1 0 0 0 
0 0 0 0 0 0 0 1 
0 1 0 0 0 0 1 0 


Predstavitev s seznami povezav ali sosedov bi lahko nadgradili z uporabo množic. Namesto v seznamu hranimo povezave ali sosede v množicah, ki so implementirane z razpršeno tabelo ali kakšno uravnotežene drevesno strukturo.

Omenjene predstavitve imajo svoje prednosti in slabosti. Primerjajmo jih med seboj glede na prostorsko zahtevnost in časovne zahtevnosti nekaterih operacij na enostavnih grafih.

<table>
    <th><td>seznam povezav</td><td>seznam sosedov</td><td>matrika sosednosti</td></th>
    <tr><td>Prostorska zahtevnost</td><td>$O(e)$</td><td>$O(n+e)$</td><td>$O(n^2)$</td></tr>
    <tr><td>Dodajanje povezave</td><td>$O(1)$</td><td>$O(1)$</td><td>$O(1)$</td></tr>
    <tr><td>Brisanje povezave</td><td>$O(e)$</td><td>$O(n)$</td><td>$O(1)$</td></tr>
    <tr><td>Dodajanje vozlišča</td><td>$O(1)$</td><td>$O(1)$</td><td>$O(n^2)$</td></tr>
    <tr><td>Brisanje vozlišča</td><td>$O(e)$</td><td>$O(e)$</td><td>$O(n^2)$</td></tr>
    <tr><td>Sosednost vozlišč</td><td>$O(e)$</td><td>$O(n)$</td><td>$O(1)$</td></tr>
    
</table>

## Preiskovanje grafov

Preiskovanje grafa (*graph traversal/search*) je sistematičen postopek, ki obišče vsa vozlišča grafa v nekem vrstnem redu. Poznamo dve pogosti vrsti preiskovanj.

### Preiskovanje v širino (*breadth-first search, BFS*)

obhod obišče najprej levo poddrevo, nato vozlišče in nato še desno poddrevo (v primeru dvojiškega drevesa). Če gre za iskalno drevo, nam vmesni obhod vrne urejeno zaporedje.

In [9]:
void BFS(int x, vector<VI> &adj, vector<int> &vis, vector<int> &seq) {
    queue<int> q;
    q.push(x); vis[x]=1;
    while (!q.empty()) {
        x=q.front(); q.pop();
        seq.push_back(x);
        for (int y : adj[x]) if (vis[y]==0) {
            q.push(y); vis[y]=1;
        }
    }
}

In [10]:
vector<int> visB(n), seqB;
BFS(0,sosedi,visB,seqB);
print(seqB);

0 1 4 3 5 7 2 6 


### Preiskovanje globino (*depth-first search, DFS*) 

obhod obišče vozlišče, nato pa rekurzivno po vrsti vsa poddrevesa.

In [11]:
void DFS(int x, vector<VI> &adj, vector<int> &vis, vector<int> &seq) {
    seq.push_back(x);
    vis[x]=1;
    for (int y : adj[x]) if (vis[y]==0) {
        DFS(y, adj, vis, seq);
    }
}

In [12]:
vector<int> visD(n), seqD;
DFS(0,sosedi,visD,seqD);
print(seqD);

0 1 3 2 5 4 7 6 


Oba opisana postopka obiščeta samo del grafa, ki je dosegljiv iz začetnega vozlišča. Tej množici vozlišč, ki so vsa povezana med seboj, rečemo **povezana komponenta** grafa (*connected component*). Za iskanje povezanih komponent lahko uporabimo kateregakoli od omenjenih postopkov za preiskovanje.

Prostorska zahtevnost obeh preiskovanj je $O(n)$. Časovno zahtevnost bi lahko ocenili z $O(n^2)$, vendar smo lahko bolj natančni z $O(e)$, ker bomo vsako povezavo obravnavali največ dvakrat (enkrat iz vsakega krajišča).

Iskanje v širino ima to lepo lastnost, da obiskuje vozlišča po nivojih od bližjih proti bolj oddaljenim. Z minimalno prilagoditvijo ga lahko uporabimo za računanje *najkrajših poti* iz začetnega vozlišča do vseh ostalih vozlišč v grafu!

Tudi iskanje v globino ima svoje lepe lastnosti. Prva je jedrnatost. Druga pa je v strukturi povezav, ki jih postopek obišče med preiskovanjem. Prehojene povezave bodo imele obliko drevesa (to sicer velja tudi za iskanje v širino). Poleg tega pa bodo vse ostale povezave v grafu vedno povezovale vozlišča z nekim svojim prednikom ali potomcem v drevesu. Nemogoče je, da bi obstajala povezava med dvema poddrevesoma. Razmislite, zakaj je temu tako. To lastnost izkoriščajo pomembni algoritmi za iskanje *mostov*, *prereznih vozlišč* in *močno povezanih komponent*.

## Topološko urejanje

Naj usmerjeni graf predstavlja medsebojne odvisnosti izvedbe opravil. Vozlišča ustrezajo opravilom, povezava $x \rightarrow y$ pa pomeni, da je treba opravilo $x$ izvesti pred opravilom $y$. V kakšnem vrstnem redu naj izvajamo opravila, da bomo lahko izvedli vsa oz. je to sploh mogoče?

<img alt="Usmerjeni graf" src="directed.svg" width="300px">

Topološki vrstni red vozlišč v usmerjenem grafu je tak vrstni red, da vse povezave v grafu kažejo od zgodnejšega proti kasnejšemu vozlišču v topološkem vrstnem redu. Topološki vrstni red ni enoličen. Za zgornji primer bi bil možen topološki vrstni red npr. `[4,0,2,3,1,6,5]`. Ker v grafu nastopa povezava $0 \rightarrow 5$, se v topološkem vrstnem redu $0$ pojavi pred $5$. Preverimo lahko, da to velja za vse povezave.

In [13]:
povezave = read_graph("directed.txt",n,m);
sosedi = adjacency_list(povezave, n, true);
for (int i=0;i<n;i++) {
    cout << i << ": ";
    print(sosedi[i]);
}

0: 1 3 5 
1: 5 
2: 3 
3: 1 6 
4: 0 3 
5: 
6: 


Razmislimo o algoritmu za izgradnjo topološkega vrstnega reda. Vozlišča brez predhodnikov lahko postavimo na začetek topološkega vrstnega reda. Če je takih vozlišč več, njihov medsebojni vrstni red ni pomemben. Za povezave, ki izhajajo iz njih, je torej poskrbljeno. Zato lahko ta vozlišča in njihove povezave odstranimo iz grafa ter ponovimo postopek z morebitnimi novimi vozlišči brez predhodnikov. Postopek se ne zaključi, če topološki vrstni red ne obstaja zaradi prisotnosti cikla v grafu. **Usmerjeni aciklični grafi** (*directed acyclic graph - DAG*) so svoj razred grafov, ki jih je mogoče topološko urediti.

Kako naj opisani postopek učinkovito implementiramo? Odstraniti moramo $n$ vozlišč in na vsakem koraku iščemo med preostalimi vozlišči kakšnega z vhodno stopnjo 0. Direktna implementacija takega postopka bo imela kvadratno časovno zahtevnost. To pa lahko izboljšamo v vodenjem seznama vozlišč z vhodno stopnjo 0. Vsakič ko odstranimo vozlišče in njegove izhodne povezave, dodamo v seznam morebitna novo nastala začetna vozlišča. Tako dobimo algoritem s časovno zahtevnostjo $O(n+e)$. Običajno je število povezav vsaj tolikšno kot število vozlišč, zato lahko brez večje škode poenostavimo na $O(e)$.

In [14]:
VI toposort(vector<VI> &sosedi, int n) {
    vector<int> indeg(n);
    for (int x=0;x<n;x++) {
        for (int y : sosedi[x]) indeg[y]++;
    }
    queue<int> q;
    for (int x=0;x<n;x++) {
        if (indeg[x]==0) q.push(x);
    }
    vector<int> seq;
    while (!q.empty()) {
        int x=q.front(); q.pop();
        seq.push_back(x);
        for (int y : sosedi[x]) {
            indeg[y]--;
            if (indeg[y]==0) q.push(y);
        }
    }
    return seq;
}

In [15]:
vector<int> topo = toposort(sosedi, n);
print(topo);

2 4 0 3 1 6 5 


## Kritična pot

Potek izvajanja projekta lahko modeliramo z mejniki in aktivnostmi, ki doprinesejo k izpolnjevanju teh mejnikov. Mejnike predstavimo z vozlišči, aktivnosti pa s povezavami v usmerjenem grafu. Ko so končane vse potrebne aktivnosti, je mejnik dosežen. Poleg tega poznamo čas $w(x,y)$ za izvedbo določene aktivnosti med mejnikoma $x$ in $y$. Očitno mora biti graf acikličen. Kakšen je najkrajši čas za izvedbo projekta ob "neomejeni" količini resursov, pri čemer lahko vsako aktivnost izvaja ena oseba, vendar imamo na voljo poljubno število oseb? Ta čas predstavlja najdaljša pot v uteženem usmerjenem acikličnem grafu, ki ji rečemo tudi kritična pot.

<img alt="utežen usmerjen acikličen graf" src="critical.svg" width="400px">

Kako pa jo izračunamo? Vozlišča naprej topološko uredimo v linearnem času. Nato pa lahko računamo najdaljše poti $d(x)$, ki se začnejo v v posameznem vozlišču $x$, v obratnem topološkem vrstnem redu. Če vozlišče nima naslednikov, je $d(x)=0$. Sicer pa velja $d(x) = \max_{y:\; x < y \,\land\, (x,y) \in E\;} (w(x,y) + d(y))$.

Opravka imamo z uteženim grafom, ki ga moramo najprej prebrati. V seznamu sosedov bomo poleg sosednjega vozlišča hranili še težo povezave, ki vodi do njega.

In [16]:
ifstream fin("critical.txt");
fin >> n >> m;
vector<VI> adj(n);
vector<VII> adjw(n);
for (int i=0;i<m;i++) {
    int a,b,c;
    fin >> a >> b >> c;
    adj[a].push_back(b);
    adjw[a].push_back({b,c});
}
fin.close();

Algoritem za izračun topološkega vrstnega reda že imamo, samo obrnemo ga.

In [20]:
vector<int> d(n);
int start = ord[0];
for (int x : ord) {
    for (auto [y,w] : adjw[x]) {
        d[x] = max(d[x], w+d[y]);
    }
    if (d[x]>d[start]) start=x;
}

cout << "dolzina = " << d[start] << endl;
cout << "kriticna pot = " << start;
int x=start;
while (d[x]!=0) {
    for (auto [y,w] : adjw[x]) {
        if (d[x]==w+d[y]) {
            cout << " " << y;
            x = y;
            break;
        }
    }
}
cout << endl;

dolzina = 10
kriticna pot = 4 6 0 5 2


## Eulerjev obhod

Dobro znan problem na neusmerjenih grafih je iskanje Eulerjevega obhod (*Eulerian tour/cycle/circuit*). Pri tem iščemo obhod, ki obišče vse povezave v grafu (vsako povezavo natanko enkrat, vozlišča pa morda tudi večkrat). Podoben problem je iskanje Eulerjevega sprehoda (*Eulerian trail/path/walk*). Pravzaprav iščemo stezo (sprehod brez ponovljenih povezav vendar morda s ponovljenimi vozlišči), ki obišče vse povezave v grafu. Za razliko od obhoda pa se lahko začne in konča na različnih mestih.

S tem problemov ste se najbrž že srečali pri risanju oblik z eno potezo (npr. odprtega pisma/ovojnice). Euler pa pri problemu sedmih mostov v Königsbergu (danes Kaliningrad). Zanimalo ga je, kako bi lahko na sprehodu prehodil vsak most natanko enkrat.

<img alt="sedem mostov v Konigsbergu (wikipedija)" src="https://upload.wikimedia.org/wikipedia/commons/5/5d/Konigsberg_bridges.png" width="200px">

Eulerjev izrek pravi, da v povezanem grafu obstaja Eulerjev obhod natanko takrat, ko so vsa vozlišča sode stopnje. Eulerjev sprehod pa natanko takrat, ko so vsa vozlišča sode stopnje razen morda točno dveh vozlišč, kjer se začne in konča. Dokažimo to trditev za primer obhoda (za sprehod velja podobno).

- Recimo, da obstaja Eulerjev obhod. Potem ta obhod na prehodu skozi vsako vozlišče zmanjša stopnjo tega vozlišča za 2. Če sproti odstranjujemo prehojene povezave, imajo na koncu vsa vozlišča stopnjo 0. Torej morajo biti na začetku vsa sode stopnje.

- Obratna smer je bolj kompleksna in jo lahko dokažemo kar s konstrukcijo Eulerjevega obhoda na povezanem grafu z vozlišči sodih stopenj. Začnemo v poljubnem vozlišču $x$ in sledimo povezavam, dokler se ne vrnemo v začetno vozlišče $x$. Pri tem se ne moremo zatakniti v nekem drugem vozlišču $y$, ker bi že porabili vse njegove povezave. V vsakem prehodu skozi vozlišče namreč porabimo dve povezavi - če je na voljo vsaj ena za vstop, bo tudi druga za izstop, ker so vsa vozlišča sode stopnje. Morda pa smo se vrnili v začetno vozlišče, pri tem pa še nismo obiskali vseh povezav. Postopek ponovimo na enem od že obiskanih vozlišč, ki ima še kakšne neobiskane povezave. Od tam na enak način zgradimo obhod in ga združimo s prejšnjim. To ponavljamo dokler niso obiskane vse povezave. To je Hierholzerjev algoritem, ki ga lahko implementiramo v linearnem času.